In [1]:
import zipfile
import io
import ast
import pandas as pd
import numpy as np
import json


In [2]:
def parse_hand(hand_text_block):
    """
    Transforme le bloc de texte d'une main
    en un dictionnaire Python propre.
    """
    hand_data = {}

    # Sépare le bloc en lignes
    for line in hand_text_block.splitlines():
        line = line.strip()
        if not line:
            continue # Ignore les lignes vides

        # Sépare la ligne en "clé = valeur"
        if ' = ' in line:
            # On sépare au premier '=' (au cas où une valeur contiendrait un '=')
            key, value_str = line.split(' = ', 1)

            # Nettoyage de la clé
            key = key.strip()

            # Nettoyage de la valeur (très important)
            value_str = value_str.strip()
            # Le fichier utilise 'true'/'false' (JS/JSON) au lieu de 'True'/'False' (Python)
            value_str = value_str.replace('true', 'True')
            value_str = value_str.replace('false', 'False')

            # Tentative de conversion de la valeur en type Python
            # (ex: "[...]" devient une liste, "10000" devient un int)
            try:
                value = ast.literal_eval(value_str)
            except (ValueError, SyntaxError):
                # Si ce n'est pas un type Python (ex: 'NT'), on garde la string
                value = value_str.strip("'") # Enlève les guillemets superflus

            hand_data[key] = value

    return hand_data


In [3]:
def parse_action(action_str):
    if action_str.startswith('d'): return None
    parts = action_str.split(' ')
    if len(parts) == 2:
        if parts[1] == 'f': return {'player_id': parts[0], 'action_type': 'FOLD', 'amount': 0}
        if parts[1] == 'cc': return {'player_id': parts[0], 'action_type': 'CALL_CHECK', 'amount': 0}
    if len(parts) == 3:
        if parts[1] == 'cbr':
            try: return {'player_id': parts[0], 'action_type': 'BET_RAISE', 'amount': int(parts[2])}
            except ValueError: return None
    return None

In [4]:
!pip install treys
from treys import Evaluator, Card
evaluator = Evaluator()

# --- NOUVEAU : Outil 2 - Normalisateur Pré-flop ---
RANKS = '23456789TJQKA'
def get_hand_key(card1, card2):
    r1, s1 = card1[0], card1[1]
    r2, s2 = card2[0], card2[1]
    rank_idx1 = RANKS.index(r1)
    rank_idx2 = RANKS.index(r2)

    if rank_idx1 > rank_idx2:
        key = r1 + r2
    else:
        key = r2 + r1

    if s1 == s2:
        if r1 != r2: key += 's'
    else:
        key += 'o'
    return key


HAND_STRENGTH_MAP = {
    "High Card": 0,
    "Pair": 1,
    "Two Pair": 2,
    "Three of a Kind": 3,
    "Straight": 4,
    "Flush": 5,
    "Full House": 6,
    "Four of a Kind": 7,
    "Straight Flush": 8
}
# --- NOUVEAU : Outil 3 - Dictionnaire d'équité Pré-flop ---
# (La table de 169 mains que nous avons définie)
PREFLOP_EQUITY_6MAX = {
    'AA': 0.648, 'KK': 0.613, 'QQ': 0.579, 'JJ': 0.546, 'TT': 0.514,
    '99': 0.482, '88': 0.451, '77': 0.421, '66': 0.391, '55': 0.362,
    '44': 0.334, '33': 0.306, '22': 0.279, 'AKs': 0.404, 'AQs': 0.383,
    'AJs': 0.369, 'ATs': 0.358, 'A9s': 0.334, 'A8s': 0.323, 'A7s': 0.312,
    'A6s': 0.300, 'A5s': 0.315, 'A4s': 0.304, 'A3s': 0.293, 'A2s': 0.283,
    'KQs': 0.370, 'KJs': 0.355, 'KTs': 0.344, 'K9s': 0.320, 'K8s': 0.297,
    'K7s': 0.286, 'K6s': 0.275, 'K5s': 0.264, 'K4s': 0.254, 'K3s': 0.244,
    'K2s': 0.235, 'QJs': 0.341, 'QTs': 0.330, 'Q9s': 0.307, 'Q8s': 0.284,
    'Q7s': 0.263, 'Q6s': 0.252, 'Q5s': 0.242, 'Q4s': 0.232, 'Q3s': 0.223,
    'Q2s': 0.214, 'JTs': 0.317, 'J9s': 0.295, 'J8s': 0.272, 'J7s': 0.252,
    'J6s': 0.232, 'J5s': 0.222, 'J4s': 0.212, 'J3s': 0.203, 'J2s': 0.195,
    'T9s': 0.284, 'T8s': 0.262, 'T7s': 0.242, 'T6s': 0.222, 'T5s': 0.204,
    'T4s': 0.194, 'T3s': 0.186, 'T2s': 0.178, '98s': 0.251, '97s': 0.232,
    '96s': 0.212, '95s': 0.194, '94s': 0.177, '93s': 0.169, '92s': 0.162,
    '87s': 0.222, '86s': 0.203, '85s': 0.185, '84s': 0.168, '83s': 0.152,
    '82s': 0.144, '76s': 0.194, '75s': 0.177, '74s': 0.160, '73s': 0.144,
    '72s': 0.137, '65s': 0.169, '64s': 0.152, '63s': 0.137, '62s': 0.130,
    '54s': 0.145, '53s': 0.130, '52s': 0.123, '43s': 0.123, '42s': 0.116,
    '32s': 0.109, 'AKo': 0.380, 'AQo': 0.358, 'AJo': 0.344, 'ATo': 0.332,
    'A9o': 0.307, 'A8o': 0.295, 'A7o': 0.283, 'A6o': 0.270, 'A5o': 0.284,
    'A4o': 0.272, 'A3o': 0.261, 'A2o': 0.250, 'KQo': 0.344, 'KJo': 0.329,
    'KTo': 0.317, 'K9o': 0.291, 'K8o': 0.267, 'K7o': 0.255, 'K6o': 0.243,
    'K5o': 0.231, 'K4o': 0.220, 'K3o': 0.210, 'K2o': 0.200, 'QJo': 0.314,
    'QTo': 0.302, 'Q9o': 0.277, 'Q8o': 0.253, 'Q7o': 0.230, 'Q6o': 0.218,
    'Q5o': 0.207, 'Q4o': 0.197, 'Q3o': 0.187, 'Q2o': 0.178, 'JTo': 0.288,
    'J9o': 0.265, 'J8o': 0.241, 'J7o': 0.219, 'J6o': 0.198, 'J5o': 0.187,
    'J4o': 0.177, 'J3o': 0.168, 'J2o': 0.159, 'T9o': 0.252, 'T8o': 0.229,
    'T7o': 0.208, 'T6o': 0.187, 'T5o': 0.169, 'T4o': 0.159, 'T3o': 0.150,
    'T2o': 0.142, '98o': 0.218, '97o': 0.198, '96o': 0.178, '95o': 0.160,
    '94o': 0.143, '93o': 0.135, '92o': 0.127, '87o': 0.187, '86o': 0.168,
    '85o': 0.150, '84o': 0.133, '83o': 0.117, '82o': 0.109, '76o': 0.159,
    '75o': 0.142, '74o': 0.125, '73o': 0.109, '72o': 0.102, '65o': 0.133,
    '64o': 0.117, '63o': 0.102, '62o': 0.095, '54o': 0.110, '53o': 0.095,
    '52o': 0.088, '43o': 0.088, '42o': 0.081, '32o': 0.074
}

In [5]:
def unroll_hand_block(hand_block_text):
    """
    Prend un bloc de texte Pluribus et le "déroule",
    retournant une liste de toutes les décisions prises pendant cette main.
    """
    decisions_in_this_hand = []

    # 1. Analyser le bloc de texte de la main
    parsed_data = parse_hand(hand_block_text)

    # Vérification : si le parsing échoue ou manque de clés, on sort
    if 'players' not in parsed_data or 'starting_stacks' not in parsed_data or 'actions' not in parsed_data:
        # print("Bloc de main corrompu, ignoré.")
        return []

    # 2. Initialiser l'état



    player_names = parsed_data['players']
    POSITION_NAMES_6MAX = ["SB", "BB", "UTG", "MP", "CO", "BTN"]
    player_position_map = {name: pos for name, pos in zip(player_names, POSITION_NAMES_6MAX)}
    position_player_map = {pos: name for name, pos in player_position_map.items()}
    current_stacks = {name: stack for name, stack in zip(player_names, parsed_data['starting_stacks'])}
    player_id_map = {f'p{i+1}': name for i, name in enumerate(player_names)}
    current_bets = {name: 0 for name in player_names}
    players_in_hand = set(player_names)
    pot_size, street, board_cards_treys = 0, 'preflop', []
    hole_cards_dict = {name: [] for name in player_names}
    treys_hole_cards = {name: [] for name in player_names}
    player_features = {name: {} for name in player_names}
    board_features = {
        'board_is_paired': 0,
        'board_is_monotone': 0,
        'board_connectedness': 0
      }
    # 3. Gérer les Blinds
    sb_amount = parsed_data['blinds_or_straddles'][0]
    bb_amount = parsed_data['blinds_or_straddles'][1]
    sb_player = player_names[0]
    bb_player = player_names[1]
    current_stacks[sb_player] -= sb_amount
    current_bets[sb_player] = sb_amount
    current_stacks[bb_player] -= bb_amount
    current_bets[bb_player] = bb_amount
    pot_size = sb_amount + bb_amount

    # 4. Boucle d'action
    for action_str in parsed_data['actions']:
        if action_str.startswith('d'):
            parts = action_str.split(' ')
            if parts[1] == 'dh': # Cartes privées
                player_id, cards_str = parts[2], parts[3]
                player_name = player_id_map.get(player_id)
                if player_name:
                    cards = [cards_str[i:i+2] for i in range(0, len(cards_str), 2)]
                    hole_cards_dict[player_name] = cards
                    treys_hole_cards[player_name] = [Card.new(c) for c in cards]
                    key = get_hand_key(cards[0], cards[1])
                    equity = PREFLOP_EQUITY_6MAX.get(key, 0.0)
                    player_features[player_name]['preflop_equity'] = equity
                    player_features[player_name]['hand_strength_rank'] = HAND_STRENGTH_MAP.get("Pair", 0) if cards[0][0] == cards[1][0] else HAND_STRENGTH_MAP.get("High Card", 0)

            elif parts[1] == 'db': # Cartes du board
                new_cards_str = parts[2]
                new_cards_list = [new_cards_str[i:i+2] for i in range(0, len(new_cards_str), 2)]
                board_cards_treys.extend([Card.new(c) for c in new_cards_list])

                if len(board_cards_treys) == 3: street = 'flop'
                elif len(board_cards_treys) == 4: street = 'turn'
                elif len(board_cards_treys) == 5: street = 'river'
                current_bets = {name: 0 for name in player_names}

                board_ranks = sorted([Card.get_rank_int(c) for c in board_cards_treys])
                board_suits = [Card.get_suit_int(c) for c in board_cards_treys]
                # 1. Paire sur le board ?
                board_features['board_is_paired'] = 1 if len(set(board_ranks)) < len(board_ranks) else 0

                # 2. Board monochrome ? (Uniquement au flop)
                if street == 'flop':
                    board_features['board_is_monotone'] = 1 if len(set(board_suits)) == 1 else 0

                # 3. Connectivité du board
                if len(board_ranks) >= 3:
                    gaps = [board_ranks[i+1] - board_ranks[i] for i in range(len(board_ranks) - 1)]
                    # 3 = "3-straight" (ex: 7-8-9), 2 = "2-gap" (ex: 7-9-J), 1 = "1-gap" (ex: 7-8-T)
                    board_features['board_connectedness'] = sum(1 for gap in gaps if gap == 1)
                    # --- MISE À JOUR DE LA FORCE + TIRAGES POUR CHAQUE JOUEUR ---
                for name in players_in_hand:
                    if treys_hole_cards[name]:
                        all_cards = treys_hole_cards[name] + board_cards_treys
                        all_ranks = sorted([Card.get_rank_int(c) for c in all_cards], reverse=True)
                        all_suits = [Card.get_suit_int(c) for c in all_cards]

                        # A. Mettre à jour la force (comme avant)
                        rank = evaluator.evaluate(treys_hole_cards[name], board_cards_treys)
                        rank_class = evaluator.get_rank_class(rank)
                        class_str = evaluator.class_to_string(rank_class)
                        player_features[name]['hand_strength_rank'] = HAND_STRENGTH_MAP.get(class_str, 0)

                        # B. Mettre à jour les tirages (uniquement Flop/Turn)
                        if street != 'river':
                            # B1. Tirage Couleur ?
                            suit_counts = [all_suits.count(s) for s in range(1, 5)]
                            player_features[name]['has_flush_draw'] = 1 if (4 in suit_counts) else 0

                            # B2. Tirage Quinte ? (Simplifié: 4 rangs uniques consécutifs ou avec un trou)
                            unique_ranks = sorted(list(set(all_ranks)))
                            player_features[name]['has_straight_draw'] = 0
                            if len(unique_ranks) >= 4:
                                for i in range(len(unique_ranks) - 3):
                                    # Vérifie un "OESD" (Open-Ended) ex: 5,6,7,8
                                    if unique_ranks[i+3] - unique_ranks[i] == 3:
                                        player_features[name]['has_straight_draw'] = 1
                                        break
                                    # Vérifie un "Gutshot" ex: 5,6,8,9
                                    if i < len(unique_ranks) - 4 and unique_ranks[i+4] - unique_ranks[i] == 4:
                                        player_features[name]['has_straight_draw'] = 1
                                        break

        else:
            parsed_action = parse_action(action_str)
            if parsed_action:
                player_id = parsed_action['player_id']
                player_name = player_id_map.get(player_id)
                if not player_name or player_name not in players_in_hand: continue

                max_bet = max(current_bets.values())
                amount_to_call = max_bet - current_bets[player_name]

                features = {
                    'hand_id': parsed_data['hand'],
                    'street': street,
                    'stack_before': current_stacks[player_name],
                    'pot_size_before': pot_size,
                    'amount_to_call': amount_to_call,
                    'players_in_hand': len(players_in_hand),
                    'hole_cards': hole_cards_dict[player_name],
                    'board_cards': [Card.int_to_str(c) for c in board_cards_treys],
                    'preflop_equity': player_features[player_name].get('preflop_equity', 0.0),
                    'hand_strength_rank': player_features[player_name].get('hand_strength_rank', 0),
                    'has_flush_draw': player_features[player_name].get('has_flush_draw', 0),
                    'has_straight_draw': player_features[player_name].get('has_straight_draw', 0),
                    'board_is_paired': board_features['board_is_paired'],
                    'board_is_monotone': board_features['board_is_monotone'],
                    'board_connectedness': board_features['board_connectedness']
                }
                for pos in POSITION_NAMES_6MAX:
                    player_name_at_pos = position_player_map.get(pos)

                    if player_name_at_pos:
                        # Un joueur est assis à ce siège
                        features[f'pos_{pos}_stack'] = current_stacks[player_name_at_pos]
                        features[f'pos_{pos}_in_hand'] = 1 if player_name_at_pos in players_in_hand else 0
                        features[f'pos_{pos}_invested'] = current_bets[player_name_at_pos]
                    else:
                        # Le siège est vide (ex: jeu à 5)
                        features[f'pos_{pos}_stack'] = 0
                        features[f'pos_{pos}_in_hand'] = 0
                        features[f'pos_{pos}_invested'] = 0

                action_type, total_bet_amount = parsed_action['action_type'], parsed_action['amount']
                target_action, invested_this_action = 'UNKNOWN', 0
                if action_type == 'FOLD':
                    target_action = 'FOLD'
                    players_in_hand.remove(player_name)
                elif action_type == 'CALL_CHECK':
                    target_action = 'CALL' if amount_to_call > 0 else 'CHECK'
                    invested_this_action = amount_to_call
                elif action_type == 'BET_RAISE':
                    invested_this_action = total_bet_amount - current_bets[player_name]
                    target_action = 'RAISE' if amount_to_call > 0 else 'BET'

                current_stacks[player_name] -= invested_this_action
                current_bets[player_name] += invested_this_action
                pot_size += invested_this_action

                features['target_action'] = target_action
                features['target_amount_invested'] = invested_this_action # On le garde pour l'instant
                decisions_in_this_hand.append(features)

    return decisions_in_this_hand

In [6]:
phh_zip = "poker-hand-histories.zip" # Votre fichier Pluribus de 1.9GB
delimiteur_de_main = "variant = 'NT'"
mains_lues = 0

all_decisions = [] # La liste finale de TOUTES les décisions

print("Démarrage de la lecture du ZIP...")

Démarrage de la lecture du ZIP...


In [7]:
try:
    with zipfile.ZipFile(phh_zip, 'r') as zf:
        for nom_fichier_interne in zf.namelist():
            if not nom_fichier_interne.endswith('.phh'):
                continue

            # print(f"Lecture de {nom_fichier_interne}...")
            with zf.open(nom_fichier_interne, 'r') as f_binaire:
                f_texte = io.TextIOWrapper(f_binaire, encoding='utf-8')
                main_actuelle_buffer = []

                for line in f_texte:
                    cleaned_line = line.strip()

                    if cleaned_line.startswith(delimiteur_de_main) and main_actuelle_buffer:
                        # 1. Traiter la main précédente
                        try:
                            hand_block = "".join(main_actuelle_buffer)
                            decisions = unroll_hand_block(hand_block)
                            if decisions:
                                all_decisions.extend(decisions)
                                mains_lues += 1
                        except Exception as e:
                            # Cette exception est maintenant beaucoup plus petite et plus sûre
                            print(f"ERREUR Critique lors du déroulage, main ignorée : {e}")

                        # 2. Réinitialiser le buffer pour la nouvelle main
                        main_actuelle_buffer = [line]

                    elif cleaned_line.startswith(delimiteur_de_main):
                        main_actuelle_buffer = [line]
                    elif main_actuelle_buffer:
                        main_actuelle_buffer.append(line)

                # --- CORRECTION DE LA DERNIÈRE MAIN ---
                # Traiter la TOUTE DERNIÈRE main du fichier
                if main_actuelle_buffer:
                    try:
                        hand_block = "".join(main_actuelle_buffer)
                        decisions = unroll_hand_block(hand_block)
                        if decisions:
                            all_decisions.extend(decisions)
                            mains_lues += 1
                    except Exception as e:
                        print(f"ERREUR Critique lors du déroulage, DERNIÈRE main ignorée : {e}")

except FileNotFoundError:
    print(f"Erreur : Le fichier '{phh_zip}' n'a pas été trouvé.")
except Exception as e:
    print(f"Une erreur générale est survenue : {e}")

print("\n--- Analyse terminée ---")
print(f"Total des mains lues et parsées : {mains_lues}")
print(f"Déroulage terminé. {len(all_decisions)} décisions créées.")


--- Analyse terminée ---
Total des mains lues et parsées : 10011
Déroulage terminé. 91444 décisions créées.


In [8]:
if all_decisions:
    ml_df = pd.DataFrame(all_decisions)
    print("\nDataFrame 'Pluribus Amélioré' créé !")

    print("\n--- 5 premières lignes (.head()) ---")
    print(ml_df.head())

    print("\n--- Vérification des nouvelles colonnes ---")
    print("\nÉquité Pré-flop (preflop_equity):")
    print(ml_df['preflop_equity'].describe())

    print("\nForce de la Main (hand_strength_rank):")
    print(ml_df['hand_strength_rank'].map({v: k for k, v in HAND_STRENGTH_MAP.items()}).value_counts(dropna=False))
else:
    print("\nERREUR : Aucune donnée n'a été créée. Vérifiez votre fichier ZIP et les délimiteurs.")


DataFrame 'Pluribus Amélioré' créé !

--- 5 premières lignes (.head()) ---
   hand_id   street  stack_before  pot_size_before  amount_to_call  \
0        0  preflop         10000              150             100   
1        0  preflop         10000              150             100   
2        0  preflop         10000              360             210   
3        0  preflop         10000              360             210   
4        0  preflop          9950              360             160   

   players_in_hand hole_cards board_cards  preflop_equity  hand_strength_rank  \
0                6   [9c, 3d]          []           0.135                   0   
1                5   [Ah, 4h]          []           0.304                   0   
2                5   [Th, 5s]          []           0.169                   0   
3                4   [6c, 7s]          []           0.159                   0   
4                3   [Tc, Qc]          []           0.330                   0   

   ...  pos_MP_i

In [9]:
ml_df.iloc[0].to_dict()

{'hand_id': 0,
 'street': 'preflop',
 'stack_before': 10000,
 'pot_size_before': 150,
 'amount_to_call': 100,
 'players_in_hand': 6,
 'hole_cards': ['9c', '3d'],
 'board_cards': [],
 'preflop_equity': 0.135,
 'hand_strength_rank': 0,
 'has_flush_draw': 0,
 'has_straight_draw': 0,
 'board_is_paired': 0,
 'board_is_monotone': 0,
 'board_connectedness': 0,
 'pos_SB_stack': 9950,
 'pos_SB_in_hand': 1,
 'pos_SB_invested': 50,
 'pos_BB_stack': 9900,
 'pos_BB_in_hand': 1,
 'pos_BB_invested': 100,
 'pos_UTG_stack': 10000,
 'pos_UTG_in_hand': 1,
 'pos_UTG_invested': 0,
 'pos_MP_stack': 10000,
 'pos_MP_in_hand': 1,
 'pos_MP_invested': 0,
 'pos_CO_stack': 10000,
 'pos_CO_in_hand': 1,
 'pos_CO_invested': 0,
 'pos_BTN_stack': 10000,
 'pos_BTN_in_hand': 1,
 'pos_BTN_invested': 0,
 'target_action': 'FOLD',
 'target_amount_invested': 0}

In [10]:
from pathlib import Path
import sys

current_dir = Path.cwd()
root_dir = current_dir.parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))
print(f"Dossier ajouté au path : {root_dir}")
# sys.path.append(str(Path(__file__).parent.parent))

from core.game_state import GameState
from features.feature_builder import FeatureExtractor

Dossier ajouté au path : /Users/killianguillaume/Desktop/RL_phase2


In [11]:
def infer_blinds_from_row(row) -> tuple[int, int]:
    """
    Déduire (small_blind, big_blind) depuis une ligne Pluribus
    
    Stratégie:
    1. Si preflop ET pos_BB_invested est "simple" → c'est la BB
    2. Sinon, chercher dans les autres colonnes (pos_SB_invested, etc.)
    3. Fallback: 50/100
    
    Args:
        row: pd.Series d'une ligne Pluribus
    
    Returns:
        (small_blind, big_blind)
    """
    
    # === Méthode 1: Depuis pos_BB_invested (preflop uniquement) ===
    if row['street'] == 'preflop':
        bb_invested = row.get('pos_BB_invested', 0)
        
        # Si BB n'a fait aucune action (juste posté la blind)
        # Heuristique: bb_invested doit être un multiple de 50
        if bb_invested > 0 and bb_invested <= 1000 and bb_invested % 50 == 0:
            big_blind = bb_invested
            small_blind = big_blind // 2
            return small_blind, big_blind
    
    # === Méthode 2: Depuis pos_SB_invested ===
    if 'pos_SB_invested' in row:
        sb_invested = row['pos_SB_invested']
        if sb_invested > 0 and sb_invested <= 500 and sb_invested % 25 == 0:
            small_blind = sb_invested
            big_blind = small_blind * 2
            return small_blind, big_blind
    
    # === Méthode 3: Déduire depuis amount_to_call ===
    # Si un joueur doit call la BB, amount_to_call ≈ BB
    if row['street'] == 'preflop' and row['amount_to_call'] > 0:
        call_amount = row['amount_to_call']
        
        # Arrondir au multiple de 50 le plus proche
        big_blind = round(call_amount / 50) * 50
        if big_blind > 0:
            small_blind = big_blind // 2
            return small_blind, big_blind
    
    # === Fallback: Valeurs par défaut ===
    return 50, 100


# === Test unitaire ===
def test_infer_blinds():
    """Tests de la fonction"""
    
    # Test 1: Preflop standard
    row1 = pd.Series({
        'street': 'preflop',
        'pos_BB_invested': 100,
        'amount_to_call': 100
    })
    assert infer_blinds_from_row(row1) == (50, 100)
    
    # Test 2: Niveau supérieur
    row2 = pd.Series({
        'street': 'preflop',
        'pos_BB_invested': 400,
        'amount_to_call': 400
    })
    assert infer_blinds_from_row(row2) == (200, 400)
    
    # Test 3: Postflop (fallback)
    row3 = pd.Series({
        'street': 'flop',
        'pos_BB_invested': 5000,
        'amount_to_call': 0
    })
    assert infer_blinds_from_row(row3) == (50, 100)
    
    print("✅ Tous les tests passés !")

# Lance le test
test_infer_blinds()


✅ Tous les tests passés !


In [12]:

def infer_player_position(row):
    """
    Déduire la position du joueur décisionnaire
    
    Logique: Le joueur décisionnaire est celui dont le stack
             correspond à stack_before
    """
    stack_before = row['stack_before']
    
    for pos in ['SB', 'BB', 'UTG', 'MP', 'CO', 'BTN']:
        col_stack = f'pos_{pos}_stack'
        col_in_hand = f'pos_{pos}_in_hand'
        
        if col_stack in row and col_in_hand in row:
            if row[col_in_hand] == 1 and row[col_stack] == stack_before:
                return pos
    
    # Fallback: si pas trouvé, retourner BTN
    return 'BTN'



In [13]:

def extract_opponent_stacks(row, hero_position):
    """
    Extraire les stacks des adversaires actifs
    """
    stacks = []
    
    for pos in ['SB', 'BB', 'UTG', 'MP', 'CO', 'BTN']:
        # Exclure le hero
        if pos == hero_position:
            continue
        
        col_stack = f'pos_{pos}_stack'
        col_in_hand = f'pos_{pos}_in_hand'
        
        if col_stack in row and col_in_hand in row:
            if row[col_in_hand] == 1:
                stacks.append(row[col_stack])
    
    return stacks


In [14]:
def calculate_pot_odds(amount_to_call, pot_size):
    """Calculer les pot odds"""
    if amount_to_call == 0:
        return 0.0
    return amount_to_call / (pot_size + amount_to_call)


In [15]:

def pluribus_row_to_gamestate(row):
    """
    Convertir une ligne Pluribus → GameState (compatible avec FeatureExtractor)
    """
    
    # 1. Identifier la position du joueur
    position = infer_player_position(row)
    
    # 2. Extraire les stacks adverses
    opponent_stacks = extract_opponent_stacks(row, position)
    small_blind, big_blind = infer_blinds_from_row(row)
    
    # 4. Calculer pot odds
    pot_odds = calculate_pot_odds(row['amount_to_call'], row['pot_size_before'])
    
    # 5. Actions légales (déduites de target_action)
    # Par défaut: fold, call toujours possibles
    legal_actions = ['fold', 'call']
    
    # Si on peut raise (stack suffisant)
    if row['stack_before'] > row['amount_to_call']:
        legal_actions.append('raise')
    'opponent_stacks'
    # 6. Construire le GameState
    gamestate = GameState(
        hole_cards=row['hole_cards'],
        board=row['board_cards'] if len(row['board_cards']) > 0 else [],
        street=row['street'],
        position=position,
        num_active_players=row['players_in_hand'],
        pot_size=row['pot_size_before'],
        stack=row['stack_before'],
        big_blind=big_blind,  # Assumé depuis les exemples
        small_blind=small_blind,
        amount_to_call=row['amount_to_call'],
        legal_actions=legal_actions,
        actions_this_street=[],  # Pas dispo dans Pluribus DF
        player_id=None,
    )
    
    return gamestate


In [16]:
ACTION_MAPPING = {
    'FOLD': 0,
    'CALL': 1,
    'CHECK': 1,   # ✅ CHECK = CALL avec montant 0
    'RAISE': 2,
    'BET': 2,     # ✅ BET = RAISE (premier à miser)
    'ALL_IN': 2   # ✅ ALL-IN = RAISE (cas extrême)
}


def map_action_to_label(action: str) -> int:
    """
    Mapper une action Pluribus → label (0, 1, 2)
    
    Raises:
        KeyError si action inconnue
    """
    action_upper = action.upper().strip()
    
    if action_upper not in ACTION_MAPPING:
        raise KeyError(f"Action inconnue: '{action}' (actions valides: {list(ACTION_MAPPING.keys())})")
    
    return ACTION_MAPPING[action_upper]


In [17]:

def convert_pluribus_df(df, output_path, max_rows=None):
    """
    Convertir le DataFrame Pluribus en features (44 dims)
    
    Args:
        df_path: Chemin vers le DataFrame Pluribus (.parquet ou .csv)
        output_path: Chemin de sortie (.npz)
        max_rows: Limiter le nombre de lignes (pour test)
    
    Returns:
        X: (N, 44) features
        y: (N,) labels (0=FOLD, 1=CALL, 2=RAISE)
    """
    
    # Charger le DataFrame
    # print(f"Chargement de {df_path}...")
    
    # if df_path.endswith('.parquet'):
    #     df = pd.read_parquet(df_path)
    # else:
    #     df = pd.read_csv(df_path)
    
    if max_rows:
        df = df.head(max_rows)
        print(f"⚠️  Limitation à {max_rows} lignes pour test")
    
    print(f"✅ DataFrame chargé: {len(df)} lignes, {len(df.columns)} colonnes")
    
    # Initialiser l'extracteur
    extractor = FeatureExtractor()
    
    # Convertir chaque ligne
    X_list = []
    y_list = []
    errors = []
    
    print("\n🔄 Conversion en cours...")
    
    for idx, row in df.iterrows():
        try:
            # 1. Convertir en GameState
            gamestate = pluribus_row_to_gamestate(row)
            
            # 2. Extraire les 44 features
            features = extractor.extract(gamestate)
            
            label = map_action_to_label(row['target_action'])
            
            X_list.append(features)
            y_list.append(label)
            
            if (idx + 1) % 10000 == 0:
                print(f"   {idx + 1:,}/{len(df):,} lignes traitées...")
        
        except Exception as e:
            errors.append((idx, str(e)))
            if len(errors) <= 5:  # Afficher les 5 premières erreurs
                print(f"⚠️  Erreur ligne {idx}: {e}")
            continue
    
    # Convertir en numpy
    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list, dtype=np.int32)
    
    print(f"\n✅ Conversion terminée!")
    print(f"   ✓ Lignes converties: {len(X):,}/{len(df):,}")
    print(f"   ✓ X shape: {X.shape}")
    print(f"   ✓ y shape: {y.shape}")
    print(f"   ✓ Erreurs: {len(errors)}")
    
    # Distribution des labels
    label_counts = np.bincount(y)
    print(f"\n📊 Distribution des actions:")
    print(f"   FOLD:  {label_counts[0]:,} ({label_counts[0]/len(y)*100:.1f}%)")
    print(f"   CALL:  {label_counts[1]:,} ({label_counts[1]/len(y)*100:.1f}%)")
    print(f"   RAISE: {label_counts[2]:,} ({label_counts[2]/len(y)*100:.1f}%)")
    
    # Sauvegarder
    print(f"\n💾 Sauvegarde dans {output_path}...")
    np.savez_compressed(output_path, X=X, y=y)
    print(f"✅ Fichier sauvegardé ({Path(output_path).stat().st_size / 1024 / 1024:.1f} MB)")
    
    return X, y

In [18]:
import argparse
from pathlib import Path
# %pip install pyarrow

# (Assure-toi d'avoir importé ta fonction convert_pluribus_df avant, 
# ou qu'elle soit définie dans une cellule précédente)
# from features.feature_builder import convert_pluribus_df 

parser = argparse.ArgumentParser(description='Convertir Pluribus DF → features')
parser.add_argument('--input', type=str, default='pluribus_processed.parquet', # J'ai ajusté le chemin car tu es déjà dans le dossier data/
                    help='Chemin du DataFrame Pluribus')
parser.add_argument('--output', type=str, default='pluribus_features.npz',
                    help='Chemin de sortie (.npz)')
parser.add_argument('--max-rows', type=int, default=None,
                    help='Limiter le nombre de lignes')

# --- C'EST ICI QUE ÇA CHANGE ---
# Pour utiliser les valeurs par défaut (default=...) :
args = parser.parse_args([]) 

# OU, pour forcer des paramètres spécifiques comme si tu étais en ligne de commande :
# args = parser.parse_args(['--max-rows', '1000', '--input', 'mon_fichier.parquet'])
# -------------------------------

print(f"📂 Input: {args.input}")
print(f"📂 Output: {args.output}")

# Créer le dossier si nécessaire (ici '.', car tu es déjà dans data/)
Path('.').mkdir(exist_ok=True) 

# Lancer la conversion (Décommente la ligne ci-dessous si la fonction est importée)
X, y = convert_pluribus_df(ml_df, args.output, args.max_rows)

print("\n🎉 Conversion terminée (Simulation)!")

📂 Input: pluribus_processed.parquet
📂 Output: pluribus_features.npz
✅ DataFrame chargé: 91444 lignes, 35 colonnes

🔄 Conversion en cours...
   10,000/91,444 lignes traitées...
   20,000/91,444 lignes traitées...
   30,000/91,444 lignes traitées...
   40,000/91,444 lignes traitées...
   50,000/91,444 lignes traitées...
   60,000/91,444 lignes traitées...
   70,000/91,444 lignes traitées...
   80,000/91,444 lignes traitées...
   90,000/91,444 lignes traitées...

✅ Conversion terminée!
   ✓ Lignes converties: 91,444/91,444
   ✓ X shape: (91444, 87)
   ✓ y shape: (91444,)
   ✓ Erreurs: 0

📊 Distribution des actions:
   FOLD:  48,313 (52.8%)
   CALL:  24,635 (26.9%)
   RAISE: 18,496 (20.2%)

💾 Sauvegarde dans pluribus_features.npz...
✅ Fichier sauvegardé (2.1 MB)

🎉 Conversion terminée (Simulation)!


In [19]:
y

array([0, 2, 0, ..., 0, 2, 1], dtype=int32)

In [20]:
# %pip install scikit-learn

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- PRÉREQUIS ---
# Assurez-vous que X_final (vos features) et Y_final (votre cible)
# sont disponibles depuis le script précédent.

# Pour le rapport final, nous avons besoin des noms de nos actions
# (l'inverse de notre 'action_map' précédente)
target_names_map = {
    0: 'FOLD',
    1: 'CHECK/CALL',
    2: 'BET/RAISE',
}
# Obtenir les noms dans le bon ordre
unique_actions = np.unique(y)
action_labels = [target_names_map[i] for i in unique_actions]

print(f"Prêt à entraîner sur {y.shape[0]} décisions et {X.shape[1]} features.")

X_train, X_test, Y_train, Y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test.shape[0]} échantillons")

rf_model = RandomForestClassifier(
    n_estimators=150,
    max_depth=30,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

# --- ÉTAPE 3 : Entraîner le modèle ---
print("\nEntraînement du Random Forest... (cela peut prendre un moment)...")
rf_model.fit(X_train, Y_train)
print("--- Entraînement terminé ! ---")

# --- ÉTAPE 4 : Évaluer le modèle ---
print("\nÉvaluation du modèle sur l'ensemble de test...")
Y_pred = rf_model.predict(X_test)

# A. Précision simple (Accuracy)
accuracy = accuracy_score(Y_test, Y_pred)
print(f"\nPrécision (Accuracy) : {accuracy * 100:.2f}%")

# B. Rapport détaillé (Beaucoup plus utile !)
# Cela montre la performance pour CHAQUE action (Fold, Call, Raise...)
print("\n--- Rapport de Classification ---")
print(classification_report(Y_test, Y_pred, target_names=action_labels))

Prêt à entraîner sur 91444 décisions et 87 features.
Taille de l'ensemble d'entraînement : 73155 échantillons
Taille de l'ensemble de test : 18289 échantillons

Entraînement du Random Forest... (cela peut prendre un moment)...
--- Entraînement terminé ! ---

Évaluation du modèle sur l'ensemble de test...

Précision (Accuracy) : 83.80%

--- Rapport de Classification ---
              precision    recall  f1-score   support

        FOLD       0.94      0.93      0.94      9663
  CHECK/CALL       0.72      0.79      0.75      4927
   BET/RAISE       0.73      0.65      0.69      3699

    accuracy                           0.84     18289
   macro avg       0.80      0.79      0.79     18289
weighted avg       0.84      0.84      0.84     18289



In [22]:
# %pip install imblearn
# %pip install optuna

In [23]:
from imblearn.over_sampling import SMOTE
# Le nouveau modèle
from xgboost import XGBClassifier
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score


/Users/killianguillaume/Desktop/RL_phase2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
smote = SMOTE(random_state=42, k_neighbors=3)

X_train_smote, Y_train_smote = smote.fit_resample(X_train, Y_train)

print("Application de SMOTE terminée.")
print(f"Nouveau set d'entraînement (après SMOTE) : {X_train_smote.shape[0]} échantillons")

# Regardez la nouvelle distribution (elle sera équilibrée !)
print("\nNouvelle distribution des classes d'entraînement :")
unique, counts = np.unique(Y_train_smote, return_counts=True)
print(dict(zip(unique, counts)))

# --- ÉTAPE 2 : Créer le modèle XGBoost ---
# XGBoost a des paramètres similaires, mais 'eval_metric' est important
xgb_model = XGBClassifier(
    n_estimators=350,       # 150 arbres séquentiels
    learning_rate=0.1,      # Vitesse d'apprentissage
    max_depth=8,           # Profondeur max (plus profond que RF)
    random_state=42,
    min_child_weight = 5,
    subsample= 0.9,
    colsample_bytree= 0.9,
    # gamma= 0.6637340904395472,
    # reg_alpha= 0.449046849267851,
    # reg_lambda= 0.9752487950663815,
    n_jobs=-1,
    use_label_encoder=False,  # Paramètres techniques
    eval_metric='mlogloss'    # Métrique d'évaluation pour multi-classes
)

# --- ÉTAPE 3 : Entraîner XGBoost sur les données SMOTE ---
print("\nEntraînement du modèle XGBoost sur les données SMOTE...")
xgb_model.fit(X_train_smote, Y_train_smote)
print("--- Entraînement terminé ! ---")

# --- ÉTAPE 4 : Évaluer sur les données de TEST (les vraies !) ---
print("\nÉvaluation du modèle XGBoost sur l'ensemble de test (original)...")
Y_pred_xgb = xgb_model.predict(X_test) # Prédit sur X_test normal

print(f"\nPrécision (Accuracy) : {accuracy_score(Y_test, Y_pred_xgb) * 100:.2f}%")

print("\n--- Rapport de Classification (XGBoost + SMOTE) ---")
print(classification_report(Y_test, Y_pred_xgb, target_names=action_labels))

Application de SMOTE terminée.
Nouveau set d'entraînement (après SMOTE) : 115950 échantillons

Nouvelle distribution des classes d'entraînement :
{0: 38650, 1: 38650, 2: 38650}

Entraînement du modèle XGBoost sur les données SMOTE...


/Users/killianguillaume/Desktop/RL_phase2/venv/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


--- Entraînement terminé ! ---

Évaluation du modèle XGBoost sur l'ensemble de test (original)...

Précision (Accuracy) : 85.46%

--- Rapport de Classification (XGBoost + SMOTE) ---
              precision    recall  f1-score   support

        FOLD       0.95      0.94      0.95      9663
  CHECK/CALL       0.74      0.81      0.78      4927
   BET/RAISE       0.75      0.67      0.71      3699

    accuracy                           0.85     18289
   macro avg       0.82      0.81      0.81     18289
weighted avg       0.86      0.85      0.85     18289



In [25]:
X_train_inner, X_val_inner, Y_train_inner, Y_val_inner = train_test_split(
    X_train, Y_train,
    test_size=0.2,
    random_state=42,
    stratify=Y_train
)


print("🔄 Application de SMOTE...")
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_smote, Y_train_smote = smote.fit_resample(X_train_inner, Y_train_inner)

print(f"✅ Avant SMOTE: {len(Y_train_inner)} samples")
print(f"✅ Après SMOTE: {len(Y_train_smote)} samples")

dtrain = xgb.DMatrix(X_train_smote, label=Y_train_smote)
dval = xgb.DMatrix(X_val_inner, label=Y_val_inner)


def objective(trial):
    """
    Fonction objectif optimisée avec XGBoost natif
    
    Améliorations vs ton code:
    - Early stopping automatique
    - Inférence plus rapide
    - Meilleur monitoring
    """
    
    # --- Hyperparamètres à optimiser ---
    params = {
        'objective': 'multi:softprob',  # ⚠️ softprob pour avoir les probabilités
        'num_class': 3,  # FOLD, CALL, RAISE
        'eval_metric': 'mlogloss',
        
        # Paramètres à optimiser
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 5.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 5.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 5.0, log=True),
        
        # Paramètres fixes
        'seed': 42,
        'tree_method': 'hist',  # ✅ Plus rapide que 'auto'
        'verbosity': 0
    }
    
    # --- Entraînement avec early stopping ---
    num_boost_round = trial.suggest_int('n_estimators', 100, 500)
    
    evals = [(dtrain, 'train'), (dval, 'val')]
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=evals,
        early_stopping_rounds=50,  # ✅ Stop si pas d'amélioration
        verbose_eval=False  # Silence pendant Optuna
    )
    
    # --- Prédiction sur validation ---
    y_pred_proba = model.predict(dval)  # Shape: (n_samples, 3)
    y_pred = np.argmax(y_pred_proba, axis=1)  # Classes prédites
    
    # --- Calcul du F1-score macro ---
    f1 = f1_score(Y_val_inner, y_pred, average='macro')
    
    # ✅ Optuna va maximiser ce score
    return f1


🔄 Application de SMOTE...
✅ Avant SMOTE: 58524 samples
✅ Après SMOTE: 92760 samples


In [26]:
# %pip install optuna

In [27]:
# print("="*70)
# print("OPTIMISATION HYPERPARAMÈTRES - XGBoost + SMOTE avec Optuna")
# print("="*70)
# optuna.logging.set_verbosity(optuna.logging.WARNING)
# study = optuna.create_study(
#     direction='maximize',
#     study_name='xgboost_smote_optimization',
#     sampler=optuna.samplers.TPESampler(n_startup_trials=50,seed=42)
# )

# # Lancer l'optimisation
# print("\n🚀 Début de l'optimisation...")
# print(f"Nombre d'essais prévus : 100")
# print("-"*70)

# study.optimize(
#     objective,
#     n_trials=100,
#     show_progress_bar=False,
#     n_jobs=1
# )


In [28]:

# # ============================================================================
# # AFFICHAGE DES RÉSULTATS
# # ============================================================================
# print("\n" + "="*70)
# print("RÉSULTATS DE L'OPTIMISATION")
# print("="*70)

# print(f"\n✅ Meilleur score (F1-macro CV) : {study.best_value:.4f}")
# print(f"📊 Nombre total d'essais : {len(study.trials)}")


# print("\n🏆 MEILLEURS HYPERPARAMÈTRES TROUVÉS :")
# print("-"*70)
# for key, value in study.best_params.items():
#     print(f"  {key:25s} : {value}")

# # ============================================================================
# # ENTRAÎNEMENT DU MODÈLE FINAL AVEC LES MEILLEURS PARAMÈTRES
# # ============================================================================
# print("\n" + "="*70)
# print("ENTRAÎNEMENT DU MODÈLE FINAL")
# print("="*70)

# # Extraction des paramètres optimaux
# best_params = study.best_params.copy()
# # Reconvertir avec les meilleurs params
# final_params = {
#     'objective': 'multi:softprob',
#     'num_class': 3,
#     'eval_metric': 'mlogloss',
#     'max_depth': best_params['max_depth'],
#     'learning_rate': best_params['learning_rate'],
#     'min_child_weight': best_params['min_child_weight'],
#     'subsample': best_params['subsample'],
#     'colsample_bytree': best_params['colsample_bytree'],
#     'gamma': best_params['gamma'],
#     'reg_alpha': best_params['reg_alpha'],
#     'reg_lambda': best_params['reg_lambda'],
#     'seed': 42,
#     'tree_method': 'hist'
# }

# # Entraîner sur TOUTES les données d'entraînement (avec SMOTE)
# final_model = xgb.train(
#     final_params,
#     dtrain,
#     num_boost_round=best_params['n_estimators'],
#     evals=[(dtrain, 'train'), (dval, 'val')],
#     early_stopping_rounds=50,
#     verbose_eval=10
# )

# print("\n📊 Évaluation sur le set de validation...")

# # Prédictions
# y_pred_proba = final_model.predict(dval)
# y_pred = np.argmax(y_pred_proba, axis=1)

# # Classification report
# print("\n" + classification_report(
#     Y_val_inner,
#     y_pred,
#     target_names=['FOLD', 'CALL', 'RAISE'],
#     digits=4
# ))



# if 'X_test' in locals() and 'Y_test' in locals():
#     print("\n🧪 Évaluation sur le set de TEST...")
    
#     dtest = xgb.DMatrix(X_test, label=Y_test)
#     y_test_pred_proba = final_model.predict(dtest)
#     y_test_pred = np.argmax(y_test_pred_proba, axis=1)
    
#     print(classification_report(
#         Y_test,
#         y_test_pred,
#         target_names=['FOLD', 'CALL', 'RAISE'],
#         digits=4
#     ))

# print("\n" + "="*60)
# print("✅ ENTRAÎNEMENT TERMINÉ")
# print("="*60)


In [29]:
# %pip install plotly

In [30]:
# %pip install nbformat

In [31]:
b

NameError: name 'b' is not defined

In [ ]:
import os
import datetime
import uuid

# Supposons que ton modèle s'appelle 'model' (ton XGBClassifier entraîné)

def save_model_with_version(model, base_name="xgb_pluribus", save_dir="models/xgb"):
    # 1. Créer le dossier s'il n'existe pas
    os.makedirs(save_dir, exist_ok=True)
    
    # 2. Générer le Timestamp (Année-Mois-Jour_Heure-Minute)
    # Ex: 2023-10-27_15-30
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    
    # 3. Générer un Hash unique court (6 caractères suffisent pour éviter les collisions)
    # Ex: a1b2c3
    unique_hash = uuid.uuid4().hex[:6]
    
    # 4. Construire le nom final
    # Ex: xgb_pluribus_2023-10-27_15-30_a1b2c3.json
    filename = f"{base_name}_{timestamp}_{unique_hash}.json"
    full_path = os.path.join(save_dir, filename)
    
    # 5. Sauvegarder
    # Note: .save_model() est la méthode native XGBoost (compatible avec ton agent C++/Python)
    model.save_model(full_path)
    
    print(f"✅ Modèle sauvegardé avec succès :")
    print(f"📁 Chemin : {full_path}")
    print(f"🔑 ID Unique : {unique_hash}")
    
    return full_path

# --- UTILISATION ---
# Une fois ton fit terminé :
# model.fit(X_train, y_train)

# Tu appelles la fonction
saved_path = save_model_with_version(xgb_model)

✅ Modèle sauvegardé avec succès :
📁 Chemin : models/xgb/xgb_pluribus_2026-01-29_12-31_fe6426.json
🔑 ID Unique : fe6426


In [ ]:
print("\n💡 Génération des visualisations Optuna...")


try:
    # Historique d'optimisation
    fig1 = plot_optimization_history(study)
    fig1.show()
    print("  ✅ Sauvegardé : optuna_optimization_history.html")

    # Importance des paramètres
    fig2 = plot_param_importances(study)
    fig2.show()
    print("  ✅ Sauvegardé : optuna_param_importances.html")

except Exception as e:
    print(f"  ⚠️ Impossible de générer les visualisations : {e}")

print("\n" + "="*70)
print("✨ OPTIMISATION TERMINÉE !")
print("="*70)

# ============================================================================
# SAUVEGARDE DU MODÈLE (optionnel)
# ============================================================================
# Décommentez pour sauvegarder le modèle
import joblib
joblib.dump(xgb_model, 'xgb_pluribus_v1.pkl')
print("\n💾 Modèle sauvegardé : xgb_pluribus_v1.pkl")


💡 Génération des visualisations Optuna...
  ⚠️ Impossible de générer les visualisations : Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

✨ OPTIMISATION TERMINÉE !



💾 Modèle sauvegardé : xgb_pluribus_v1.pkl
